In [ ]:
import pandas as pd
import os


# Merge variables

In [ ]:

# Define paths
data1_path = r"D:\File_auto\0_UCL_CASA\OneDrive - University College London\Xiaoyi_dissertation\Analysis\Data_output\NO_weighted"
data2_path = r"D:\File_auto\0_UCL_CASA\OneDrive - University College London\Xiaoyi_dissertation\Analysis\Data\dfl_traffic_count"
data3_path = r"D:\File_auto\0_UCL_CASA\OneDrive - University College London\Xiaoyi_dissertation\Analysis\Data\tfl_crowding_data"

# Function to read CSV files containing "Camden" in their names
def read_camden_csvs(folder_path):
    camden_dfs = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv') and 'Camden' in file:
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)
                camden_dfs.append(df)
    return camden_dfs

# Read data1 and data2
data1_dfs = read_camden_csvs(data1_path)
data2_dfs = read_camden_csvs(data2_path)

# Merge data1 and data2 on 'Date' and 'count_data'
merged_data1_data2 = pd.DataFrame()
for df1 in data1_dfs:
    for df2 in data2_dfs:
        merged_df = pd.merge(df1, df2, left_on='Date', right_on='count_data', how='left')
        merged_data1_data2 = pd.concat([merged_data1_data2, merged_df], ignore_index=True)

# Read data3 and filter by 'Borough' == 'Camden'
data3_dfs = []
for year in range(2019, 2025):
    file_name = f"filtered_StationFootfall_{year}_merged.csv"
    file_path = os.path.join(data3_path, file_name)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        camden_df = df[df['Borough'] == 'Camden']
        data3_dfs.append(camden_df)

# Merge data3 with merged_data1_data2 on 'TravelDate'
final_merged_data = merged_data1_data2.copy()
for df3 in data3_dfs:
    final_merged_data = pd.merge(final_merged_data, df3, left_on='Date', right_on='TravelDate', how='left')

# Save the final merged data to a CSV file
output_path = r"D:\File_auto\0_UCL_CASA\OneDrive - University College London\Xiaoyi_dissertation\Analysis\Data_output\Correlation\corr_merged.csv"
final_merged_data.to_csv(output_path, index=False)

print("Data merged and saved successfully.")